In [100]:
import pandas as pd
import torch
from torch import nn
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from transformers import AdamW
from tqdm import tqdm
from tqdm import trange
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score
import pandas as pd
from tqdm import tqdm

In [2]:
import numpy as np

In [3]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [195]:
df_train = pd.read_csv('./drive/MyDrive/VK_hack/data_final_mapping_lenta.csv').iloc[:, 1:]

<ipython-input-195-80a236771624>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('./drive/MyDrive/VK_hack/data_final_mapping_lenta.csv').iloc[:, 1:]


In [196]:
df_train.shape

(202474, 23)

In [197]:
extra_topics_llm_gen = pd.read_csv('./drive/MyDrive/VK_hack/extra_topics.csv').iloc[:, 1:]
weather_articles = pd.read_csv('./drive/MyDrive/VK_hack/weather_extra.csv').iloc[:, 1:]

In [198]:
df_train.loc[:, 'X'] = df_train.loc[:, 'title'] + ' ' + df_train.loc[:, 'text']

In [199]:
# df_train_og = df_train[~df_train.X.isna()]
df_train = df_train_og.copy()

In [200]:
weather_articles.head()

,text,tag
0,В Волгограде из-за жаркой погоды раньше времен...,Погода
1,Татарстанцы в ночь с 19 на 20 апреля наблюдали...,Погода
2,Полярное сияние заметили над Челябинской облас...,Погода
3,Под Петербургом «расцвели» уникальные ледяные ...,Погода
4,"Фонтаны в Петергофе запустили, несмотря на сне...",Погода


In [29]:
label_map = {
    'Красота': 0,
    'Политика': 1,
    'Экономика': 2,
    'Мода': 3,
    'Общество': 4,
    'Закон и право': 5,
    'Кино': 6,
    'Телевидение': 7,
    'Персоны': 8,
    'События': 9,
    'Бренды': 10,
    'Наука': 11,
    'Гаджеты': 12,
    'Соцсети': 13,
    'Дом': 14,
    'Технологии': 15,
    'Транспорт': 16,
    'Погода': 17,
    'Рецепты': 18,
    'Опросы': 19,
    'Головоломки': 20
}

In [202]:
for i, row in weather_articles.iterrows():
    break

### Data augmentation

In [203]:
df_train.loc[:, 'Погода'] = 0

for i, row in weather_articles.iterrows():
    df_train.loc[len(df_train), :] = 0
    df_train.loc[len(df_train), 'Погода'] = 1
    df_train.loc[len(df_train), 'Погода'] = 1

    df_train.loc[len(df_train), 'X'] = row.text

In [204]:
extra_topics_llm_gen.head()

,Погода,Рецепты,Опросы,Головоломки
0,\nПогода и её влияние на нашу жизнь - это тема...,\n# Рецепты\n## Вкусные лепешки со сливочным с...,"\nОпросы - это способ изучения предпочтений, м...","\n«Мистерия» - головоломки, которые удивят люб..."
1,"\n""Погода в нашем регионе последние недели оче...","\n""Новинки из мира ресторанов! Узнайте новые р...",\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!...,"\nЗагадок и головоломок, которые вызывают у ва..."
2,\nТекущее положение погоды на всей земной пове...,\nУжасно простой рецепт для начинающих кулинар...,\n```\nОпросы: Каким образом влияют опросы на ...,\nУчиться решать головоломки - это не только и...
3,\nПогода в нашем городе - неотъемлемая часть н...,\nЗдорово! Самая лучшая европейская кухня на с...,\n«Теневые тенденции: что говорят опросы?»\n\n...,"\nВсегда в поисках новых головоломок, чтобы по..."
4,"\nПогода - это основный фактор, который опреде...","\nНазвание: ""Вкусные и здоровые рецепты для вс...","\nУзнайте, насколько важна для нас работа в оф...","\nТеперь мы знаем, что головоломки - это не то..."


In [205]:
# Погода

vals = extra_topics_llm_gen.Погода.str.replace('\n', '').str.replace('!', '')
vals = vals[vals!='']
extra_df = pd.DataFrame(columns = df_train.columns, index=range(len(vals)))
extra_df.loc[:, 'Погода'] = 1
extra_df.loc[:, 'X'] = vals
extra_df = extra_df.fillna(0)

In [206]:
extra_df.shape

(762, 25)

In [207]:
df_train = pd.concat([df_train, extra_df], axis=0)

In [208]:
df_train.shape

(203316, 25)

In [209]:
df_train.loc[:, 'Рецепты'] = 0
df_train.loc[:, 'Опросы'] = 0
df_train.loc[:, 'Головоломки'] = 0

In [210]:
# Рецепты
vals = extra_topics_llm_gen.Рецепты.str.replace('\n', '').str.replace('!', '')
vals = vals[vals!='']
extra_df = pd.DataFrame(columns = df_train.columns, index=range(len(vals)))
extra_df.loc[:, 'Рецепты'] = 1
extra_df.loc[:, 'X'] = vals
extra_df = extra_df.fillna(0)
df_train = pd.concat([df_train, extra_df], axis=0)

# Опросы
vals = extra_topics_llm_gen.Опросы.str.replace('\n', '').str.replace('!', '')
vals = vals[vals!='']
extra_df = pd.DataFrame(columns = df_train.columns, index=range(len(vals)))
extra_df.loc[:, 'Опросы'] = 1
extra_df.loc[:, 'X'] = vals
extra_df = extra_df.fillna(0)
df_train = pd.concat([df_train, extra_df], axis=0)

# Головоломки
vals = extra_topics_llm_gen.Головоломки.str.replace('\n', '').str.replace('!', '')
vals = vals[vals!='']
extra_df = pd.DataFrame(columns = df_train.columns, index=range(len(vals)))
extra_df.loc[:, 'Головоломки'] = 1
extra_df.loc[:, 'X'] = vals
extra_df = extra_df.fillna(0)
df_train = pd.concat([df_train, extra_df], axis=0)

In [211]:
df_train.shape

(205579, 28)

### Label stratify

In [31]:
# Function to get indexes of columns with 1s
def get_indexes(row):
    return [i for i, val in enumerate(row) if val == 1]

# Apply the function to each row
labels = df_train.loc[:, label_map.keys()].apply(get_indexes, axis=1)

NameError: name 'df_train' is not defined

In [213]:
df_train[(df_train.Экономика == 1) & (df_train.Телевидение == 1)].iloc[0:1]

,url,title,text,topic,tags,date,Красота,Политика,Экономика,Мода,...,Гаджеты,Соцсети,Дом,Технологии,Транспорт,X,Погода,Рецепты,Опросы,Головоломки
66,https://lenta.ru/news/2013/01/14/arab/,В рекламе колонок использовали события «арабск...,В рекламе колонок Logitech EU использовали соб...,Интернет и СМИ,Реклама,2013/01/14,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,В рекламе колонок использовали события «арабск...,0.0,0,0,0


In [240]:
# df_train.loc[:, 'labels'] = labels
df_train.loc[:, 'labels'] = df_train.loc[:, 'labels'].astype(str)

In [241]:
df_train.shape

(205539, 29)

In [242]:
df_train = df_train[df_train.labels != '[]']

In [243]:
df_train.shape

(205539, 29)

In [224]:
df_train.to_csv('./drive/MyDrive/VK_hack/final_bert_train_df.csv', index=False)

In [244]:
# Function for custom sampling
def custom_sampling(group):
    if len(group) > 40_000:
        return group.sample(frac=0.25, replace=False, random_state=1)
    elif len(group) < 40_000 and len(group) >= 20_000:
        return group.sample(frac=0.35, replace=False, random_state=1)
    elif len(group) < 20_000 and len(group) >= 10_000:
        return group.sample(frac=0.65, replace=False, random_state=1)
    elif len(group) < 10_000 and len(group) >= 3_000:
        return group.sample(frac=0.8, replace=False, random_state=1)
    else:
        return group  # Sample 4 observations for groups with more than 4 observations

# Apply custom sampling to each group
sampled_df = df_train.groupby('labels').apply(custom_sampling)

In [245]:
labels = sampled_df.labels

In [246]:
df_train.labels.value_counts()

labels
[4, 8]        41769
[1, 5]        40716
[2]           22246
[5]           20420
[4, 5]        19825
[7]           11797
[6, 7]        10720
[13]           9319
[11]           7932
[14]           5180
[12, 15]       3706
[8, 9]         3664
[0, 3, 10]     3613
[17]            802
[19]            763
[18]            751
[20]            749
[16]            514
[2, 7]          443
[5, 8]          437
[6]             173
Name: count, dtype: int64

In [247]:
sampled_df.labels.value_counts()

labels
[4, 5]        12886
[4, 8]        10442
[1, 5]        10179
[2]            7786
[7]            7668
[13]           7455
[5]            7147
[6, 7]         6968
[11]           6346
[14]           4144
[12, 15]       2965
[8, 9]         2931
[0, 3, 10]     2890
[17]            802
[19]            763
[18]            751
[20]            749
[16]            514
[2, 7]          443
[5, 8]          437
[6]             173
Name: count, dtype: int64

In [248]:
indxes = np.arange(len(labels))
tr_indx, _ = train_test_split(indxes, test_size=0.2, random_state=42 , stratify=labels)

In [250]:
train_ready_df = sampled_df.copy()
# labels = labels.iloc[tr_indx]

In [293]:
train_ready_df = train_ready_df[~train_ready_df.X.isna()]

In [294]:
train_ready_df.to_csv('./drive/MyDrive/VK_hack/final_bert_sampled_train_df.csv', index=False)

## Train

In [139]:
train_ready_df = pd.read_csv('./drive/MyDrive/VK_hack/df_extra_gen_old.csv')

In [140]:
train_ready_df.loc[train_ready_df.loc[:, 'X'].isna(), 'X'] = train_ready_df.loc[train_ready_df.loc[:, 'X'].isna(), 'text']

In [141]:
train_ready_df.loc[:, 'text'].isna().sum()

0

In [142]:
train_ready_df.loc[:, 'X'].isna().sum()

0

In [143]:
train_ready_df.actual_labels.value_counts()

actual_labels
[4, 5]           12886
[4, 8]           10442
[1, 5]           10179
[2]               7786
[7]               7667
[13]              7455
[5]               7147
[6, 7]            6968
[11]              6346
[14]              4144
[12, 15]          2965
[8, 9]            2931
[0, 3, 10]        2890
[16]               902
[17]               780
[19]               763
[18]               754
[20]               753
[2, 7]             443
[5, 8]             437
[12, 13]           400
[14, 15]           400
[6, 8]             400
[1, 2]             400
[2, 12, 15]        400
[11, 16]           380
[6]                173
[1, 4, 9, 17]       16
[4, 9]               9
[6, 19]              9
[1, 6, 19]           8
[5, 10]              5
[4, 9, 17]           3
Name: count, dtype: int64

In [144]:
train_ready_df.loc[:, 'clean_X'] = train_ready_df['X'].str.replace('\n', ' ').str.replace('!', ' ')

In [151]:
train_ready_df.X[train_ready_df.X.str.len() < 200].values[10]

'Температура и влажность: как они влияют на жизнь населенных пунктов'

In [146]:
train_ready_df.iloc[672].X

'Одну из\xa0самых больших модных ошибок перестали считать ошибкой Одна из самых распространенных «модных ошибок» — ношение сандалий с носками — стала популярным трендом. Об этом сообщает Footwear News. Модные эксперты утверждают, что при правильном сочетании расцветки носков с открытой обувью эта комбинация может выглядеть стильно. Они также дали несколько советов, какие именно носки лучше всего подойдут к сандалиям. Например, под женственные сандалии на высоком каблуке или платформе рекомендуется выбирать свободные прозрачные носки. Под спортивные сандалии или тапки вроде Birkenstocks подойдут гольфы. Лучше всего это сочетание будет смотреться с шортами-бермудами. К однотонным сандалиям спокойных цветов (например, серебристого, белого или коричневого) советуют подбирать яркие цветные носки. При этом одними из наиболее удачных оттенков назвали красный и оранжевый. Носки с узорами подойдут к цветным сандалиям. Правильно сочетать эти вещи достаточно сложно, однако специалисты предложили 

In [147]:
train_ready_df.shape

(97241, 32)

In [148]:
clean_train_ready_df = train_ready_df[train_ready_df.loc[:, 'clean_X'] != ' ']

In [149]:
clean_train_ready_df.shape

(97241, 32)

In [59]:
labels = clean_train_ready_df.loc[:, label_map.keys()].apply(get_indexes, axis=1)
clean_train_ready_df.loc[:, 'cut_labels'] = labels
clean_train_ready_df.loc[:, 'cut_labels'] = clean_train_ready_df.loc[:, 'cut_labels'].astype(str)
clean_train_ready_df = clean_train_ready_df[clean_train_ready_df.loc[:, 'cut_labels'] != '[4, 17]']
labels = clean_train_ready_df.loc[:, label_map.keys()].apply(get_indexes, axis=1)

<ipython-input-59-7330a5c8a878>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_train_ready_df.loc[:, 'cut_labels'] = labels


In [60]:
clean_train_ready_df.shape

(97241, 32)

In [61]:
sentences = clean_train_ready_df.clean_X.to_list()

In [62]:
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")

In [63]:
# max_len = 0

# lengths = []
# # For every sentence...
# for sent in tqdm(sentences):

#     # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
#     input_ids = tokenizer.encode(sent, add_special_tokens=True)

#     # Update the maximum sentence length.
#     lengths.append(len(input_ids))

100%|██████████| 97241/97241 [02:45<00:00, 587.46it/s]


In [65]:
labels_t = torch.tensor(clean_train_ready_df.loc[:, label_map.keys()].values)

In [66]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in tqdm(sentences):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

  0%|          | 0/97241 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 97241/97241 [04:10<00:00, 387.72it/s]


In [67]:
# Convert the lists into tensors.
input_ids_t = torch.cat(input_ids, dim=0)
attention_masks_t = torch.cat(attention_masks, dim=0)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[1])
print('Token IDs:', input_ids_t[1])

Original:  Бренд Z Zegna вдохновился лыжниками 1970-х годов Итальянская марка Z Zegna представила новую коллекцию одежды. Презентация прошла в рамках международной выставки мужской моды Pitti Uomo, которая проходит во Флоренции. Об этом корреспонденту «Ленты.ру» сообщил представитель пресс-службы бренда. Источником вдохновения для создания одежды стали образы лыжников 1970-х годов. «Мы создали вещи, которые отвечают активному образу жизни современных людей, подчеркивая при этом роскошь и эстетику», — сказал креативный директор марки Алессандро Сартори (Alessandro Sartori). Основные цвета коллекции — черный, белый, темно-зеленый и горчичный. Z Zegna — вторая линия бренда Ermenegildo Zegna, принадлежащего одноименной итальянской компании. Компания занимается выпуском люксовой мужской одежды, обуви и парфюмерии. Была основана в 1910 году Эрменеджильдо Зенья. Сейчас управляется уже четвертым поколением семьи Зенья.
Token IDs: tensor([    2, 71858,    62,  3690,  6241, 33763,  9754, 77827, 

In [68]:
labels_t.shape

torch.Size([97241, 21])

In [69]:
indxes = np.arange(len(labels))
tr_indx, vl_indx = train_test_split(indxes, test_size=0.2, random_state=42 , stratify=labels)

In [89]:
np.save('./drive/MyDrive/VK_hack/tr_indx.npy', tr_indx)
np.save('./drive/MyDrive/VK_hack/vl_indx.npy', vl_indx)

In [71]:
clean_train_ready_df.to_csv('./drive/MyDrive/VK_hack/df_extra_gen_old.csv', index=False)

In [72]:
labels[0]

[0, 3, 10]

In [73]:
from torch.utils.data import TensorDataset, Subset

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids_t, attention_masks_t, labels_t)

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = Subset(dataset, tr_indx), Subset(dataset, vl_indx)

In [74]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it
# here. For fine-tuning BERT on a specific task, the authors recommend a batch
# size of 16 or 32.
batch_size = 8

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [75]:
mapping_all = {v: k for k, v in label_map.items()}

In [76]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained(
    "cointegrated/rubert-tiny2", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(mapping_all.keys()), # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.
    output_attentions = False, # Whether the model returns attentions weights.
    problem_type = "multi_label_classification",
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [77]:
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [78]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4.
# We chose to run for 2,I have already seen that the model starts overfitting beyound 2 epochs
epochs = 7

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [79]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.argmax(axis=1)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [80]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [81]:
device = 'cuda'

In [82]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, 

In [83]:
model.dropout.p = 0.2

In [84]:
for name, param in model.named_parameters():
    if "classifier" not in name and "pooler" not in name:  # Check if the parameter belongs to the last layer
        param.requires_grad = False

In [85]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

bert.pooler.dense.weight
bert.pooler.dense.bias
classifier.weight
classifier.bias


In [86]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 101

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss,
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 1000 batches.
        if step % 1000 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because
        # accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # are given and what flags are set. For our usage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        out = model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels.float())
        (loss, logits) = out.loss, out.logits

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            out = model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels.float())
            (loss, logits) = out.loss, out.logits

        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)


    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time() - total_t0)))


======== Epoch 1 / 7 ========
Training...
  Batch 1,000  of  9,724.    Elapsed: 0:00:16.
  Batch 2,000  of  9,724.    Elapsed: 0:00:28.
  Batch 3,000  of  9,724.    Elapsed: 0:00:40.
  Batch 4,000  of  9,724.    Elapsed: 0:00:50.
  Batch 5,000  of  9,724.    Elapsed: 0:01:00.
  Batch 6,000  of  9,724.    Elapsed: 0:01:14.
  Batch 7,000  of  9,724.    Elapsed: 0:01:29.
  Batch 8,000  of  9,724.    Elapsed: 0:01:42.
  Batch 9,000  of  9,724.    Elapsed: 0:01:52.

  Average training loss: 0.14
  Training epcoh took: 0:01:59

Running Validation...
  Accuracy: 0.55
  Validation Loss: 0.10
  Validation took: 0:00:18

======== Epoch 2 / 7 ========
Training...
  Batch 1,000  of  9,724.    Elapsed: 0:00:10.
  Batch 2,000  of  9,724.    Elapsed: 0:00:20.
  Batch 3,000  of  9,724.    Elapsed: 0:00:31.
  Batch 4,000  of  9,724.    Elapsed: 0:00:41.
  Batch 5,000  of  9,724.    Elapsed: 0:00:51.
  Batch 6,000  of  9,724.    Elapsed: 0:01:01.
  Batch 7,000  of  9,724.    Elapsed: 0:01:12.
  Batch 8

In [93]:
model.eval()

# Tracking variables
total_probs = []
total_labels = []

# Evaluate data for one epoch
for batch in validation_dataloader:

    # Unpack this training batch from our dataloader.
    #
    # As we unpack the batch, we'll also copy each tensor to the GPU using
    # the `to` method.
    #
    # `batch` contains three pytorch tensors:
    #   [0]: input ids
    #   [1]: attention masks
    #   [2]: labels
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():

        # Forward pass, calculate logit predictions.
        # token_type_ids is the same as the "segment ids", which
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        out = model(b_input_ids,
                                token_type_ids=None,
                                attention_mask=b_input_mask,
                                labels=b_labels.float())
        (loss, logits) = out.loss, out.logits

    # Accumulate the validation loss.
    total_eval_loss += loss.item()

    # Move logits and labels to CPU
    probs = torch.sigmoid(logits.detach().cpu()).numpy()
    label_ids = b_labels.to('cpu').numpy()
    total_probs.append(probs)
    total_labels.append(label_ids)

In [95]:
total_probs_ = np.concatenate(total_probs, axis=0)
total_labels_ = np.concatenate(total_labels, axis=0)

In [99]:
total_preds = np.where(total_probs_ > 0.5, 1, 0)

In [110]:
total_preds

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [101]:
f1_micro = f1_score(total_labels_, total_preds, average='micro')
f1_marco = f1_score(total_labels_, total_preds, average='macro')

In [107]:
dict(f1_micro=f1_micro, f1_marco=f1_marco)

{'f1_micro': 0.7661203913509336, 'f1_marco': 0.7790725834960351}

In [112]:
## with xgboost

In [111]:
y_tst_pred = np.load('./drive/MyDrive/VK_hack/xgb_preds.npy')

In [114]:
total_preds = y_tst_pred + total_preds
total_preds = np.where(total_preds == 0, 0, 1)

In [115]:
f1_micro = f1_score(total_labels_, total_preds, average='micro')
f1_marco = f1_score(total_labels_, total_preds, average='macro')

In [116]:
dict(f1_micro=f1_micro, f1_marco=f1_marco)

{'f1_micro': 0.8112931491872599, 'f1_marco': 0.8406146735716169}

In [108]:
# # classification_report(total_labels_, total_preds)
# from sklearn.metrics import confusion_matrix

# cm = confusion_matrix(total_labels_, total_preds)

# print("Confusion Matrix:")
# print(cm)

In [102]:
f1_micro, f1_marco

(0.7661203913509336, 0.7790725834960351)

In [109]:
tokenizer.save_pretrained('./drive/MyDrive/VK_hack/bert_ft/tokenizer/')
model.save_pretrained('./drive/MyDrive/VK_hack/bert_ft/')

In [ ]:
# Пример входных данных для классификации
input_text = "'Организация по безопасности и сотрудничеству в Европе (ОБСЕ) раскритиковала законопроект председателя комитета по безопасности Госдумы Ирины Яровой, предусматривающий уголовную ответственность за критику войск антигитлеровской коалиции. Об этом сообщается в пресс-релизе ОБСЕ. Как утверждает представитель международной организации по свободе СМИ Дуня Миятович (Dunja Mijatović), проект Яровой несет потенциальную угрозу свободе слова и прессы, а в конечном счете может повредить демократии и соблюдению прав граждан. Миятович призвала российские власти пересмотреть предложенный закон, отметив, что он «выходит за рамки борьбы с прославлением нацизма». При этом у российских властей уже достаточно законных средств для борьбы с оправданием нацистских преступлений, считает представитель ОБСЕ. По словам Миятович, общество имеет право знать о различных точках зрения и спорных случаях в истории, «даже если они неприятны или имеют провокационный характер». Как подчеркивается в пресс-релизе ОБСЕ, информировать общество о подобных разногласиях  должны именно СМИ. Кроме того, Миятович отметила, что новое предложение Яровой вызывает особое беспокойство с учетом принятия в России запрета на пропаганду гомосексуальных отношений и закона об оскорблении чувств верующих. По мнению ОБСЕ, эти нормы также могут повредить свободе прессы и свободному обмену информацией. Ирина Яровая 24 июня внесла на рассмотрение Госдумы законопроект, который предполагает изменения в главу УК «Преступления против мира и безопасности человечества». Новая статья, как передает ИТАР-ТАСС, получит название «Реабилитация нацизма» и будет предусматривать наказание за отрицание приговора Нюрнбергского трибунала и критику армий антигитлеровской коалиции. В качестве наказания Яровая предлагает ввести штраф в размере до 300 тысяч рублей либо лишение свободы до трех лет. Если действия, предусмотренные законопроектом, будут совершены с использованием служебного положения или СМИ, виновные понесут наказание в виде штрафа от 100 до 500 тысяч рублей или тюремного срока до пяти лет без права занимать определенные должности до трех лет.'"

# Токенизация текста
input_ids = tokenizer.encode(input_text, add_special_tokens=True, max_length=128, truncation=True, padding='max_length', return_tensors='pt')

# Запуск модели на входных данных
with torch.no_grad():
    outputs = model(input_ids)

# Получение предсказаний класса
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()

print("Predicted class:", predicted_class)


In [ ]:
# Пример входных данных для классификации
input_text = 'В Москве, в здании Российского фонда культуры с 11 часов утра проходила церемония прощания с Раисой Горбачевой, которая была одним из основателей Фонда. Организацию гражданской панихиды взяло на себя управлениеделами президента России. Во вторник гроб с телом супругибывшего президента СССР был на спецсамолете доставлен в Москву из Германии, где Раиса Горбачева скончалась в госпитале города Мюнстер утром 20 сентября. Весь день у гроба покойной дежурили родственники, соратники и близкие друзья четы Горбачевых. В их числе -Александр Яковлев и Вадим Медведев, Георгий Шахназаров, Никита Михалков и другие. Сам Михаил Горбачев сидел у изголовья гроба, рядом сним находились дочь Ирина и две внучки. Как сообщили РИА "Новости" близкие экс-президента, состояние  здоровья и эмоциональное состояние Михаил Сергеевича  "очень неважные".В 10.30 - еще до начала церемонии прощания - в здание Российского фонда культуры прибыл премьер-министр РоссииВладимир Путин. Он долго беседовал с Михаилом Горбачевым. Тогда же прибыли венки от президента и правительства страны. Около пятнадцати минут провела в здании Российскогофонда культуры Наина Ельцина. Наина Иосифовна выразила Михаилу Горбачеву и его родным соболезнования от имени всей семьи Ельциных.Затем, как сообщает корреспондент РИА "Новости", супругапрезидента России, выглядевшая очень опечаленной, покинулаздание Российского фонда культуры, не считая возможным общение с прессой. Пресс-секретарь президента России Дмитрий Якушкин сообщил журналистам, что сам Борис Ельцин работаетсегодня в резиденции Горки-9. Дань памяти покойной отдали экс-премьер РоссииЕвгений Примаков и бывший мэр С.-Петербурга Анатолий Собчак. На похороны ожидается прибытие Маргарет Тэтчер и ГельмутаКоля. Поступили соболезнования от канцлера ФРГ Герхарда Шредера,президента Франции Жака Ширака, президента США Билла Клинтона,других государственных и общественных деятелей. Папа РимскийИоанн Павел II молится за упокой души Раисы Горбачевой инаправил личную телеграмму соболезнования в связи с ее кончинойМихаилу Горбачеву. В течение всего дня от станции метро "Кропоткинская" до здания Фонда на Гоголевском бульваре стоит длинная очередь желающих почтить ее память. Большинство людей несут цветы. Для соблюдения порядкаприлегающие к зданию улицы были оцеплены милицией и ограждены. Отстанции метро "Кропоткинская" людей пропускают в здание небольшими группами. C учетом большого желающих проститься, церемония, которая должна была завершится в 16.00, была продлена на час и завершилась в  17.00. Завтра с 10.30 до 11.30 в здании Российского фондакультуры пройдет гражданская панихида. Ее организацию взяло насебя Управление делами президента России. Завтра же днемсостоятся похороны Раисы Горбачевой на Новодевичьем кладбище.'
# Токенизация текста
input_ids = tokenizer.encode(input_text, add_special_tokens=True, max_length=128, truncation=True, padding='max_length', return_tensors='pt')

# Запуск модели на входных данных
with torch.no_grad():
    outputs = model(input_ids)

# Получение предсказаний класса
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()

print("Predicted class:", predicted_class)

In [ ]:
'В Ольхонском районе Иркутской области обнаружили палаточный детский лагерь, работавший без разрешительных документов. Об этом в понедельник, 27 июня, сообщается на сайте регионального главка МВД. Отмечается, что лагерь «Солнечный берег», находившийся на берегу Байкала, работал без разрешения оздоровительной комиссии, созданной при правительстве области. Сотрудники полиции обнаружили нарушения во время заезда отдыхающих. «В лагере отсутствовали видеонаблюдение, физическая охрана, кнопка тревожной сигнализации, ограждения, подобранный персонал не был проверен по полицейским учетам. Установлены и другие недостатки», — говорится в сообщении главка. Из 77 прибывших на отдых несовершеннолетних, 35 перевели в другой лагерь, а остальных передали родителям. В следственном региональном управлении СКР добавили, что проводится проверка по факту оказания услуг, не отвечающих требованиям безопасности. «Роспотребнадзор лагерю не выдал разрешение на открытие, так как в водоеме рядом с лагерем была обнаружена кишечная палочка. Персонал уверяет, что для приготовления пищи использовалась привозная вода, а вода из водоема использовалась только для хозяйственных нужд», — уточнили в управлении. 23 июня сотрудники Следственного комитета возбудили уголовное дело по факту нарушений, выявленных в незарегистрированном детском лагере «Золото Белого моря» на острове Сидоров в Карелии. Установлено, что там не соблюдались санитарно-эпидемиологические нормы. По данным прокуратуры, руководство лагеря произвело самозахват территории и самовольно возвело постройки, в которых находились дети. Внеплановые проверки детских лагерей в Карелии, Псковской, Архангельской, Ленинградской, Калининградской и Новгородской областях начались 20 июня. За два дня до этого в Карелии погибли 14 детей, отдыхавших в лагере «Парк-отель Сямозеро». Они отправились вместе с инструкторами на водную прогулку и утонули во время шторма.'